In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Getting only 1k from the training portion of data
train_images = train_images[:1000]
train_labels = train_labels[:1000]

In [ ]:
# Getting only 1k from the testing portion of data
test_images = test_images[:1000]
test_labels = test_labels[:1000]

In [ ]:
# Reshape the images to include the channel dimension (since EfficientNet expects 3D input)
train_images = tf.expand_dims(train_images, axis=-1)
test_images = tf.expand_dims(test_images, axis=-1)
print(train_images.shape)  # (1000, 28, 28, 1)
print(test_images.shape)   # (1000, 28, 28, 1

In [ ]:
# Resize images to match EfficientNet input size (224x224) and convert to 3 channels
train_images = tf.image.grayscale_to_rgb(train_images)
train_images = tf.image.resize(train_images, (224, 224))

test_images = tf.image.grayscale_to_rgb(test_images)
test_images = tf.image.resize(test_images, (224, 224))

In [ ]:
print(train_images.shape)  # (1000, 224, 224, 3)
print(test_images.shape)   # (1000, 224, 224, 3)

In [ ]:
# Normalize images to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [ ]:
# Load the EfficientNetB0 model, excluding the top classification layers
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                            input_shape=(224, 224, 3))

# Freeze the base model to retain its pre-trained weights
base_model.trainable = False

In [ ]:
# Create a custom top layer
model = models.Sequential([
    base_model,
    # Converts the feature maps into a single vector
    layers.GlobalAveragePooling2D(),
    # Add a fully connected layer with 128 neurons
    layers.Dense(128, activation='relu'),
    # Final layer with 10 outputs for 10 classes
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, epochs=5,
                    validation_data=(test_images,
                    test_labels))

In [ ]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_images,
												 train_labels,
												 epochs=5,
												 validation_data=(test_images[:1000],
																					test_labels[:1000]
																					)
												)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images[:1000],
        test_labels[:1000])

print(f"Test accuracy: {test_accuracy * 100:.2f}")